# RDS Lab 6

Today we will be talking about a privacy-preserving way to create synthetic data that (optionally) retains some useful characteristics of the original dataset. You can read about the DataSynthesizer in [Ping, Stoyanovich, and Howe (2017)](https://faculty.washington.edu/billhowe/publications/pdfs/ping17datasynthesizer.pdf)

## Import DataSynthesizer

The original code can be downloaded from [Github](https://github.com/DataResponsibly/DataSynthesizer), but we have it set up on Jupyter Hub for today. 

In [ ]:
import os, sys
# Add the direcotry of DataSynthesizer into sys.path before importing the code
instructor_path = os.getcwd() + '/../../shared/Lab 6/DataSynthesizer/DataSynthesizer'
student_path = os.getcwd() + '/../shared/Lab 6/DataSynthesizer/DataSynthesizer'
path = student_path # change this during lab!
sys.path.append(path)

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import Image

In [ ]:
from DataDescriber import DataDescriber
from DataGenerator import DataGenerator
from ModelInspector import ModelInspector
from lib.utils import read_json_file

In [ ]:
# Intialize a describer and a generator
describer = DataDescriber()
generator = DataGenerator()

## Preparation


In [ ]:
# Set up some paths
sensitive_data_file = path + '/../data/adult_reduced.csv'
description_files = {'random mode':                   'description(random).json', 
                     'independent attribute mode':    'description(independent).json', 
                     'correlated attribute mode':     'description(correlated).json'}
synthetic_data_files = {'random mode':                'synthetic data(random).csv', 
                        'independent attribute mode': 'synthetic data(independent).csv', 
                        'correlated attribute mode':  'synthetic data(correlated).csv'}


output_data_size = 1000

In [ ]:
# Read in the data
real_data = pd.read_csv(sensitive_data_file)
real_data.head()

## Random mode

In random mode, we replace the feature that we want to protect with random values drawn from a uniform distribution

In [ ]:
describer.describe_dataset_in_random_mode(sensitive_data_file)

In [ ]:
describer.save_dataset_description_to_file(description_files['random mode'])

#### Important parameters here

- __seed__: initialize the random number generator for both `random` and `np.random`
- __mininum__, __maxmimum__: determines the min and max of the random distribution from which we will draw the new values

In [ ]:
generator.generate_dataset_in_random_mode(n=output_data_size, 
                                          description_file=description_files['random mode'], 
                                          seed=0, 
                                          minimum=0, 
                                          maximum=100)

In [ ]:
generator.save_synthetic_data(synthetic_data_files['random mode'])

In [ ]:
synthetic_random = pd.read_csv(synthetic_data_files['random mode'])

In [ ]:
real_data.head()

In [ ]:
synthetic_random.head()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(8,3), dpi=100)
axes[0].hist(real_data['age'])
axes[1].hist(synthetic_random['age'])
axes[0].set_xlabel('age')
axes[1].set_xlabel('age')
axes[0].set_title('real data')
axes[1].set_title('random mode');

In [ ]:
relationship_real = real_data['relationship'].value_counts()
relationship_random = synthetic_random['relationship'].value_counts()
relationship_both = pd.merge(relationship_real.to_frame(), 
                             relationship_random.to_frame(), 
                             left_index=True, 
                             right_index=True, 
                             suffixes=('_real', '_random_mode'))
relationship_both

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(6,3), dpi=120)
axes[0].bar(relationship_both.index, relationship_both['relationship_real'])
axes[1].bar(relationship_both.index, relationship_both['relationship_random_mode'])
axes[0].set_title('real data')
axes[1].set_title('random mode')
fig.autofmt_xdate()

In [ ]:
with open(description_files['random mode']) as f:
    description_json = json.load(f)

In [ ]:
description_json.keys()

In [ ]:
print(json.dumps(description_json['attribute_description']['relationship'], indent=4))

## Independent attribute mode

In independent attribute mode, we replace the feature we want to protect with values that follow the same distribution as the original data. However, we do not consider any other features when determining the new value for the sensitive feature. 

In [ ]:
describer.describe_dataset_in_independent_attribute_mode(sensitive_data_file, epsilon=0.1)

In [ ]:
describer.save_dataset_description_to_file(description_files['independent attribute mode'])

In [ ]:
generator.generate_dataset_in_independent_mode(n=output_data_size, 
                                               description_file=description_files['independent attribute mode'], 
                                               seed=0)

In [ ]:
generator.save_synthetic_data(synthetic_data_files['independent attribute mode'])

In [ ]:
synthetic_independent = pd.read_csv(synthetic_data_files['independent attribute mode'])

In [ ]:
synthetic_independent.head()

In [ ]:
# Plot age before and after being protected using indepenent attribute mode
fig, axes = plt.subplots(1, 2, figsize=(8,3), dpi=100)
axes[0].hist(real_data['age'])
axes[1].hist(synthetic_independent['age'])
axes[0].set_xlabel('age')
axes[1].set_xlabel('age')
axes[0].set_title('real data')
axes[1].set_title('independent attribute mode');

In [ ]:
# Plot the relationship between two variables to see if changes after protecting age
real_data.groupby(['income']).age.plot(kind="hist",  title='real data', legend=True)
plt.show()
synthetic_independent.groupby(['income']).age.plot(kind="hist",  title='independent attribute mode', legend=True)
plt.show()


In [ ]:
relationship_real = real_data['relationship'].value_counts()
relationship_independent = synthetic_independent['relationship'].value_counts()
relationship_both = pd.merge(relationship_real.to_frame(), 
                             relationship_independent.to_frame(), 
                             left_index=True, 
                             right_index=True, 
                             suffixes=('_real', '_independent_attribute_mode'))

fig, axes = plt.subplots(1, 2, figsize=(6,3), dpi=120)
axes[0].bar(relationship_both.index, relationship_both['relationship_real'])
axes[1].bar(relationship_both.index, relationship_both['relationship_independent_attribute_mode'])
axes[0].set_title('real data')
axes[1].set_title('independent attribute mode')
fig.autofmt_xdate()

## Correlated attribute mode

The correlated attribute mode replaces the feature we want to protect with values based on the *conditional* distribution of all the features in the database. This is calculated using a Bayesian network.

In [ ]:
describer.describe_dataset_in_correlated_attribute_mode(sensitive_data_file, 
                                                        epsilon=0.1, 
                                                        k=2)

In [ ]:
from lib.utils import display_bayesian_network

In [ ]:
display_bayesian_network(describer.bayesian_network)

In [ ]:
describer.save_dataset_description_to_file(description_files['correlated attribute mode'])

In [ ]:
generator.generate_dataset_in_correlated_attribute_mode(n=output_data_size, 
                                                        description_file=description_files['correlated attribute mode'],
                                                        seed=0)

In [ ]:
generator.save_synthetic_data(synthetic_data_files['correlated attribute mode'])

In [ ]:
synthetic_correlated = pd.read_csv(synthetic_data_files['correlated attribute mode'])

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(8,3), dpi=100)
axes[0].hist(real_data['age'])
axes[1].hist(synthetic_correlated['age'])
axes[0].set_xlabel('age')
axes[1].set_xlabel('age')
axes[0].set_title('real data')
axes[1].set_title('correlated attribute mode');

In [ ]:
# Plot the relationship between two variables to see if changes after protecting age
real_data.groupby(['income']).age.plot(kind="hist",  title='real data', legend=True)
plt.show()
synthetic_correlated.groupby(['income']).age.plot(kind="hist",  title='correlated attribute mode', legend=True)
plt.show()


In [ ]:
relationship_real = real_data['relationship'].value_counts()
relationship_correlated = synthetic_correlated['relationship'].value_counts()
relationship_both = pd.merge(relationship_real.to_frame(), 
                             relationship_correlated.to_frame(), 
                             left_index=True, 
                             right_index=True, 
                             suffixes=('_real', '_correlated_attribute_mode'))

fig, axes = plt.subplots(1, 2, figsize=(6,3), dpi=120)
axes[0].bar(relationship_both.index, relationship_both['relationship_real'])
axes[1].bar(relationship_both.index, relationship_both['relationship_correlated_attribute_mode'])
axes[0].set_title('real data')
axes[1].set_title('correlated attribute mode')
fig.autofmt_xdate()

## Statistical measures

### Mutual information

We can use mutual information to further understand how the relationships between features are similar/different in the real data and the synthetic data. Mutual information is defined as follows for two discrete variables X and Y:


$$I(X; Y) = \sum_{y \in Y} \sum_{x \in X} p(x,y) \log(\frac{p(x,y)}{p(x)p(y)})$$

Higher values indicate greater levels of mutual information. For two independent variables, the value will be zero (look at the logged term). This metric works for categorical variables *or* continuous variables. 

In [ ]:
from sklearn.metrics import normalized_mutual_info_score

In [ ]:
normalized_mutual_info_score(real_data['marital-status'], 
                             real_data['relationship'], 
                             average_method='arithmetic')

In [ ]:
normalized_mutual_info_score(real_data['marital-status'], 
                             real_data['education'], 
                             average_method='arithmetic')

We can even plot the mutual information between all pairs of features in the dataset.

In [ ]:
attribute_description = read_json_file(description_files['correlated attribute mode'])['attribute_description']
ModelInspector(real_data, 
               synthetic_correlated, 
               attribute_description).mutual_information_heatmap()

### Two-sample Kolmogorov–Smirnov test

The Kolmogorov-Smirnov test quantifies the similarity between the empirical distribution functions two sets of data. We can use it to measure how well the distribution of our synthetic data mimics the distribution of the original dataset. 

For two empirical distributions P and Q, the Kolmogorov–Smirnov test statistic is defined as:

$$D = \max_x |P(x) - Q(x)|$$

Intuitively, think of plotting the CDF of each dataset. The Kolmogorov–Smirnov test statistic is the maxium distance between the two CDFs. 

<img src="https://upload.wikimedia.org/wikipedia/commons/c/cf/KS_Example.png">

This metric only works for continuous variables. 

In [ ]:
from scipy.stats import ks_2samp

In [ ]:
def ks_test(df_in: pd.DataFrame, df_out: pd.DataFrame, attr: str):
    """
    df_in: the sensitive dataset
    df_out: the synthetic dataset
    attr: the attribute that will be calculated for Two-sample Kolmogorov–Smirnov test.
    """
    return ks_2samp(df_in[attr], df_out[attr])[0]

In [ ]:
ks_test(real_data, synthetic_random, 'age')

In [ ]:
ks_test(real_data, synthetic_independent, 'age')

In [ ]:
ks_test(real_data, synthetic_correlated, 'age')

### KL-divergence

The KL-divergence is an alternative way to measure the difference between two distributions. For two distributions P and Q, the KL-divergence is defined as 

$$D = \sum_x P(x) \log (\frac{P(x)}{Q(X)})$$

This metric works on discrete/categorical variables.


In [ ]:
from scipy.stats import entropy

In [ ]:
def get_distribution_of_categorical_attribute(attribute: pd.Series, indicies=None):
    distribution = attribute.dropna().value_counts()
    if indicies is not None:
        for idx in set(indicies) - set(distribution.index):
            distribution.loc[idx] = 0
    distribution.sort_index(inplace=True)
    return distribution/sum(distribution)

def kl_test(df_in: pd.DataFrame, df_out: pd.DataFrame, attr: str):
    """
    df_in: the sensitive dataset
    df_out: the synthetic dataset
    attr: the attribute that will be calculated for KL-divergence.
    """
    distribution_in = get_distribution_of_categorical_attribute(df_in[attr])
    distribution_out = get_distribution_of_categorical_attribute(df_out[attr], distribution_in.index)
    return entropy(distribution_out, distribution_in)

In [ ]:
kl_test(real_data, synthetic_random, 'relationship')

In [ ]:
kl_test(real_data, synthetic_independent, 'relationship')

In [ ]:
kl_test(real_data, synthetic_correlated, 'relationship')

### Boxplot

In [ ]:
import seaborn as sns

In [ ]:
real_data.head()

In [ ]:
plt.figure(dpi=100)
sns.boxplot(x='income', y='age', data=real_data, linewidth=0.5, fliersize=2);